<a href="https://colab.research.google.com/github/AnudeepReddy-Katta/nlp-journey/blob/main/LSTM/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchtext.legacy import data 
# The torchtext package consists of data processing utilities and popular datasets for natural language.

In [2]:
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic  = True 
# To have reproducability internally

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  include_lengths = True) # Ability to define a preprocessing pipeline

LABEL = data.LabelField(dtype=torch.float)

In [3]:
from torchtext.legacy import datasets
# The raw text iterators for common NLP datasets

In [4]:
train_data, test_data = datasets.IMDB.splits(TEXT,LABEL)

In [5]:
print(len(train_data))
print(len(test_data))

25000
25000


In [6]:
print(vars(train_data.examples[0]))

{'text': ['Who', 'would', "n't", 'want', 'to', 'go', 'on', 'road', 'trip', 'with', 'Peter', 'Falk', '?', 'That', 'guy', "'s", 'right', 'eye', 'has', 'more', 'character', 'than', 'most', 'actors', 'today', '.', 'This', 'is', 'the', 'kind', 'of', 'funny', 'and', 'touching', 'movie', 'we', 'are', 'all', 'looking', 'for', 'as', 'a', 'counterbalance', 'to', 'all', 'the', 'bombastic', 'special', 'effects', 'bores', '.', 'Women', 'are', 'going', 'to', 'love', 'it', 'for', 'all', 'the', 'wake', '-', 'up', 'romance', 'advice', 'for', 'men', ',', 'and', 'men', 'will', 'love', 'it', 'for', 'its', 'spot', '-', 'on', 'father', '/', 'son', 'character', 'study', '--', 'one', 'great', 'little', 'scene', 'after', 'another', '.', 'And', 'it', 'has', 'just', 'enough', 'of', 'an', 'edge', 'to', 'be', 'a', 'true', 'indie', 'find', '.', 'Obviously', 'this', 'is', 'a', 'labor', 'of', 'love', 'for', 'Paul', 'Reiser', 'who', 'understands', 'what', 'it', "'s", 'like', 'to', 'be', 'both', 'a', 'father', 'and', '

In [7]:
# Train-valid data split

import random 

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [8]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

17500
7500
25000


In [9]:
# Building vocab

MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [10]:
print(len(TEXT.vocab))
print(len(LABEL.vocab))

25002
2


In [11]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202995), (',', 192589), ('.', 166220), ('a', 109823), ('and', 109605), ('of', 100631), ('to', 94086), ('is', 76261), ('in', 61475), ('I', 54572), ('it', 53798), ('that', 49288), ('"', 44123), ("'s", 43366), ('this', 42618), ('-', 36817), ('/><br', 35673), ('was', 35263), ('as', 30534), ('with', 29957)]


In [12]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [13]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [14]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    # iterator that batches examples of similar lengths together
    (train_data,valid_data,test_data),
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device
)

In [15]:
import torch.nn as nn

In [16]:
# RNN class

class LSTM(nn.Module):

  def __init__(self,num_embeddings,embedding_dim,hidden_dim,output_dim):
    
    super().__init__()

    self.embedding = nn.Embedding(num_embeddings,embedding_dim) 
    self.lstm = nn.LSTM(embedding_dim,hidden_dim)
    self.fc = nn.Linear(hidden_dim,output_dim)

  def forward(self,text,text_length):

    #[sentence len, batch size] => [sentence len, batch size, embedding size]
    embedded = self.embedding(text)

    packed = torch.nn.utils.rnn.pack_padded_sequence(embedded,text_length.to('cpu'))

    #[sentence len, batch size, embedding size] => 
    #  output: [sentence len, batch size, hidden size]
    #  hidden: [1, batch size, hidden size]
    packed_output, (hidden,cell) = self.lstm(packed)
    return self.fc(hidden.squeeze(0)).view(-1)

In [17]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [18]:
model

LSTM(
  (embedding): Embedding(25002, 128)
  (lstm): LSTM(128, 256)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

In [19]:
def count_parameters(model):

  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [20]:
count_parameters(model)

3595777

In [21]:
import torch.optim as optim

In [22]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [23]:
criterion = nn.BCEWithLogitsLoss() # Sigmoid + BCE Loss

In [24]:
model = model.to(device)
criterion = criterion.to(device)

In [25]:
def binary_accuracy(preds,y):
  
  rounded_preds = torch.round(torch.sigmoid(preds))
  correct = (rounded_preds == y).float()
  acc = correct.sum() / len(correct)

  return acc

In [26]:
def train(model, iterator, optimizer, criterion):

  epoch_loss = 0
  epoch_acc = 0

  model.train()

  for batch in iterator:

    text, text_length = batch.text
    optimizer.zero_grad()

    logits = model(text, text_length)

    loss = criterion(logits, batch.label)
    acc = binary_accuracy(logits, batch.label)

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()
    epoch_acc += acc.item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [27]:
def evaluate(model, iterator, criterion):

  epoch_loss = 0
  epoch_acc = 0

  model.eval()

  with torch.no_grad():

    for batch in iterator:

      text, text_length = batch.text
      logits = model(text, text_length)

      loss = criterion(logits, batch.label)
      acc = binary_accuracy(logits, batch.label)

      epoch_loss += loss.item()
      epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [28]:
import time

def epoch_time(start_time, end_time):

  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time/60)
  elapsed_secs = elapsed_time - elapsed_mins

  return elapsed_mins, elapsed_secs

In [29]:
N_EPOCHS = 30

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

  start_time = time.time()
    
  train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
      
  end_time = time.time()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  if valid_loss < best_valid_loss:

    best_valid_loss = valid_loss
    torch.save(model.state_dict(),'model.pt')

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 7.525287628173828s
	Train Loss: 0.689 | Train Acc: 53.60%
	 Val. Loss: 0.684 |  Val. Acc: 57.03%
Epoch: 02 | Epoch Time: 0m 7.174832820892334s
	Train Loss: 0.659 | Train Acc: 61.52%
	 Val. Loss: 0.600 |  Val. Acc: 67.53%
Epoch: 03 | Epoch Time: 0m 7.233463525772095s
	Train Loss: 0.601 | Train Acc: 68.47%
	 Val. Loss: 0.599 |  Val. Acc: 67.05%
Epoch: 04 | Epoch Time: 0m 7.230190992355347s
	Train Loss: 0.545 | Train Acc: 73.10%
	 Val. Loss: 0.536 |  Val. Acc: 73.48%
Epoch: 05 | Epoch Time: 0m 7.30656623840332s
	Train Loss: 0.505 | Train Acc: 76.31%
	 Val. Loss: 0.501 |  Val. Acc: 76.04%
Epoch: 06 | Epoch Time: 0m 7.294423818588257s
	Train Loss: 0.471 | Train Acc: 78.66%
	 Val. Loss: 0.483 |  Val. Acc: 77.17%
Epoch: 07 | Epoch Time: 0m 7.323093414306641s
	Train Loss: 0.447 | Train Acc: 80.13%
	 Val. Loss: 0.492 |  Val. Acc: 77.39%
Epoch: 08 | Epoch Time: 0m 7.340047836303711s
	Train Loss: 0.523 | Train Acc: 75.83%
	 Val. Loss: 0.460 |  Val. Acc: 79.05%
Epoch: 09

In [30]:
model.load_state_dict(torch.load('model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.355 | Test Acc: 85.25%
